## Prepare Standardized Precipitation Evapotranspiration Index (SPEI)

In this notebook, the 3-month SPEI for Germany from 1981 to 2024 is calculated.
Steps include:
- Calculate the Potential Evatotranspiration from ERA5-land data (2m temperature, 10 m wind speed, surface net solar radiation and relative humidity)
- Calculate the SPEI using the water balance (total precipitation minus potential evapotranspiration)
- Crop everything to Germany and save as yearly datasets
- Also crop the SPEI downloaded from the official SPEI database to Germany and save as yearly datasets to have a comparison to the calculated values

In [1]:
# import libraries
import os
import xarray as xr
import geopandas as gpd
import odc.stac
import rasterio
import pyet
import numpy as np
import pandas as pd
import xclim.indicators.atmos

In [2]:
# set working directory
os.chdir("E:/Master/Thesis/3_Data")
print("Current working directory: {0}".format(os.getcwd()))

Current working directory: E:\Master\Thesis\3_Data


In [3]:
# prepare input datasets for calculating PET
# open era5 dataset
ds = xr.open_dataset("./Raw/ERA5-Land/data_1.nc", decode_coords="all",decode_times=True)

# rename dimensions to fit other data
ds = ds.rename({"valid_time":"time",
                            "longitude": "lon",
                            "latitude": "lat"})

In [4]:
# extract temperature data
t2m = ds.drop_vars(["number", "expver", "u10", "v10"])

# convert temperature from K to °C (required for pyet package)
t2m['t2m'] = t2m.t2m - 273.15

# change unit in variable attributes
t2m.t2m.attrs["units"] = "°C"

# assign crs to data
t2m = t2m.rio.write_crs('WGS 1984')

In [5]:
# create new variable that combines u and v component of wind
# source: https://knowledge.dea.ga.gov.au/notebooks/How_to_guides/External_data_ERA5_Climate/
ds["wind10m"] = (ds["u10"]**2 + ds["v10"]**2)**0.5

# extract wind data
wind = ds.drop_vars(["number", "expver", "u10", "v10", "t2m"])

# set unit in variable attributes
wind.wind10m.attrs["units"] = "m/s"

# assign crs to data
wind = wind.rio.write_crs('WGS 1984')

In [6]:
# open era5 surface net solar radiation dataset
ssr = xr.open_dataset("./Raw/ERA5-Land/data_4.nc", decode_coords="all",decode_times=True)

# rename dimensions to fit other data
ssr = ssr.rename({"valid_time":"time",
                            "longitude": "lon",
                            "latitude": "lat"})

# convert radiation from J to MJ (required for pyet package)
ssr['ssr'] = ssr.ssr / 1000000

# change unit in variable attributes
ssr.ssr.attrs["units"] = "MJ/m²"

# assign crs to data
ssr = ssr.rio.write_crs('WGS 1984')

In [7]:
# open era5 surface pressure dataset
sp = xr.open_dataset("./Raw/ERA5-Land/data_5.nc", decode_coords="all",decode_times=True)

# rename dimensions to fit other data
sp = sp.rename({"valid_time":"time",
                            "longitude": "lon",
                            "latitude": "lat"})

# convert pressure from Pa to kPa (required for pyet package)
sp['sp'] = sp.sp / 1000

# change unit in variable attributes
sp.sp.attrs["units"] = "kPa"

# assign crs to data
sp = sp.rio.write_crs('WGS 1984')

In [8]:
# open era5 2 m dewpoint temperature dataset
d2m = xr.open_dataset("./Raw/ERA5-Land/data_6.nc", decode_coords="all",decode_times=True)

# rename dimensions to fit other data
d2m = d2m.rename({"valid_time":"time",
                            "longitude": "lon",
                            "latitude": "lat"})

# convert temperature from K to °C (required for pyet package)
d2m['d2m'] = d2m.d2m - 273.15

# change unit in variable attributes
d2m.d2m.attrs["units"] = "°C"

# assign crs to data
d2m = d2m.rio.write_crs('WGS 1984')

In [9]:
# calculate relative humidity from mean temperature and mean dewpoint temperature
# source: equation 8 from https://journals.ametsoc.org/view/journals/bams/86/2/bams-86-2-225.xml?tab_body=pdf , dissolved for RH
# RH ≈ 100 × e^((17.625 × Td) / (243.04 + Td) - (17.625 × T) / (243.04 + T))

rh = 100 * np.exp((17.625 * d2m.d2m) / (243.04 + d2m.d2m) - (17.625 * t2m.t2m) / (243.04 + t2m.t2m))

# set unit in variable attributes
rh.attrs["units"] = "%"

In [10]:
# load and prepare precipitation data from era5-land
# the data has ben split into two while downloading
filelist = ("./Raw/ERA5-Land/data_2.nc", "./Raw/ERA5-Land/data_3.nc")
era5 = xr.open_mfdataset(filelist, concat_dim="valid_time", combine='nested')

# rename dimensions to fit other data
era5 = era5.rename({"valid_time":"time",
                            "longitude": "lon",
                            "latitude": "lat"})

# sort by date
era5 = era5.sortby("time")

# extract precipitation data
tp = era5.drop_vars(["number", "expver"])

# convert precipitation from "m of water equivalent per day" to "mm / month"
# source: ECMWF conversion table https://confluence.ecmwf.int/pages/viewpage.action?pageId=197702790
tp['tp'] = tp.tp * 1000 * tp.tp.time.dt.days_in_month

# change unit in variable attributes
tp.tp.attrs["units"] = "mm/month"

# assign crs to data
tp = tp.rio.write_crs('WGS 1984')

In [11]:
# load Germany shapefile 
ger = gpd.read_file("./GER.shp")

In [12]:
# extract one date to create mask of germany with dimensions of climate data (all era5-land datasets share same dimensions)
wind_ex = wind.sel(time = "1981-08-01", method = "nearest")

In [13]:
# create mask of research area using the dimensions of the exemplary climate data
ger_mask = rasterio.features.geometry_mask(ger.geometry, 
                                            out_shape=wind_ex.odc.geobox.shape,
                                            transform=wind_ex.odc.geobox.affine,
                                            all_touched=False,
                                            invert=False)

In [14]:
ger_mask = xr.DataArray(ger_mask, dims=("lat", "lon"))

In [15]:
# apply mask of research area to datasets
t2m_ger = t2m["t2m"].where(~ger_mask)
wind_ger = wind["wind10m"].where(~ger_mask)
ssr_ger = ssr["ssr"].where(~ger_mask)
sp_ger = sp["sp"].where(~ger_mask)
rh_ger = rh.where(~ger_mask)
tp_ger = tp["tp"].where(~ger_mask)

In [16]:
# calculate potential evapotranspiration 
pet = pyet.pm_fao56(tmean = t2m_ger, wind = wind_ger, rn = ssr_ger, rh = rh_ger, pressure = sp_ger)

In [17]:
# set unit in variable attributes
pet.attrs["units"] = "mm/month"

In [18]:
# calculate water balance (precipitation - potential evapotranspiration)
wb = tp_ger - pet

In [19]:
wb.attrs["units"] = "mm/month"

In [20]:
# calculate spei with xclim
spei = xclim.indicators.atmos.standardized_precipitation_evapotranspiration_index(wb = wb, 
                                                                                  freq = "MS", # monthly resampling with a 3-month moving window
                                                                                  window = 3, 
                                                                                  dist = "gamma")

c:\Users\irish\miniconda3\envs\Thesis_Msc\Lib\site-packages\xclim\indices\stats.py:975: UserWarning: The input data is chunked on time dimension and must be fully rechunked to run `fit` on groups . Beware, this operation can significantly increase the number of tasks dask has to handle.
  da, _ = preprocess_standardized_index(da, freq=freq, window=window, **indexer)


In [21]:
spei

<xarray.DataArray 'spei' (time: 528, lat: 83, lon: 98)> Size: 34MB
dask.array<where, shape=(528, 83, 98), dtype=float64, chunksize=(1, 83, 98), chunktype=numpy.ndarray>
Coordinates:
  * time         (time) datetime64[ns] 4kB 1981-01-01 1981-02-01 ... 2024-12-01
  * lat          (lat) float64 664B 55.2 55.1 55.0 54.9 ... 47.3 47.2 47.1 47.0
  * lon          (lon) float64 784B 5.45 5.55 5.65 5.75 ... 14.95 15.05 15.15
    spatial_ref  int64 8B 0
    number       int64 8B 0
    expver       (time) <U4 8kB dask.array<chunksize=(528,), meta=np.ndarray>
Attributes: (12/13)
    calibration_period:  ('1981-01-01', '2024-12-01')
    freq:                MS
    window:              3
    scipy_dist:          gamma
    method:              ML
    group:               time.month
    ...                  ...
    time_indexer:        {}
    cell_methods:        
    history:             [2025-05-14 11:02:17] spei: xclim.core.indicator.SPE...
    standard_name:       spei
    long_name:           Standardized precipitation evapotranspiration index ...
    description:         Water budget (precipitation minus evapotranspiration...

In [ ]:
# save calculated spei dataset
# compress data to reduce size
comp = dict(zlib=True, complevel=4)
spei.encoding.update(comp)

# save as netCDF4 dataset
spei.to_netcdf("./SPEI/SPEI_calc_3M_GER.nc")

In [24]:
# for comparison, also quickly process the SPEI downloaded from the official SPEI database
spei_db = xr.open_dataset("./Raw/spei03.nc", decode_coords="all",decode_times=True)

In [25]:
spei_db

<xarray.Dataset> Size: 2GB
Dimensions:  (lon: 720, lat: 360, time: 1476)
Coordinates:
  * lon      (lon) float64 6kB -179.8 -179.2 -178.8 -178.2 ... 178.8 179.2 179.8
  * lat      (lat) float64 3kB -89.75 -89.25 -88.75 -88.25 ... 88.75 89.25 89.75
  * time     (time) datetime64[ns] 12kB 1901-01-16 1901-02-15 ... 2023-12-16
    crs      int32 4B ...
Data variables:
    spei     (time, lat, lon) float32 2GB ...
Attributes: (12/15)
    conventions:  CF-1.11
    title:        Global 3-months SPEI, z-values, 0.5 degree
    version:      2.10.0
    id:           ./outputNcdf/spei03.nc
    summary:      Global dataset of the Standardized Precipitation-Evapotrans...
    keywords:     drought, climatology, SPEI, Standardized Precipitation-Evap...
    ...           ...
    software:     Created in R using the SPEI package (https://cran.r-project...
    call:         spei.nc(sca=i, inPre=./inputData/cru_ts4.08.1901.2023.pre.d...
    date:         Wed Jul 10 09:02:28 2024
    reference:    Beguería S., Vicente-Serrano S., Reig F., Latorre B. (2014)...
    reference2:   Vicente-Serrano S.M., Beguería S., López-Moreno J.I. (2010)...
    reference3:   Beguería S., Vicente-Serrano S., Angulo-Martínez M. (2010) ...

In [26]:
# drop all values before 1981
spei_db = spei_db.sel(time=slice("1981-01-01", None))

In [27]:
# extract one date to create mask of germany with dimensions of spei data
spei_db_ex = spei_db.sel(time = "2023-12-16", method = "nearest")

In [28]:
# create mask of research area using the dimensions of the exemplary data
ger_mask = rasterio.features.geometry_mask(ger.geometry, 
                                            out_shape=spei_db_ex.odc.geobox.shape,
                                            transform=spei_db_ex.odc.geobox.affine,
                                            all_touched=False,
                                            invert=False)

In [29]:
ger_mask = xr.DataArray(ger_mask, dims=("lat", "lon"))

In [30]:
# apply mask of research area to dataset
spei_db_ger = spei_db["spei"].where(~ger_mask)

In [31]:
# compress data to reduce size
comp = dict(zlib=True, complevel=4)
spei_db_ger.encoding.update(comp)

# save as netCDF4 dataset
spei_db_ger.to_netcdf("./SPEI/SPEI_database_3M_GER.nc")